In [1]:
from dotenv import load_dotenv
import os

# Laden Sie die Umgebungsvariablen aus der .env-Datei
load_dotenv()
API_KEY = os.environ.get("API_KEY")

In [2]:
# # load vectorDB
# with open("faiss_store.pkl", "rb") as f:
#     vectorstore = pickle.load(f)

In [3]:
import faiss
import pickle
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.docstore.in_memory import InMemoryDocstore


# Load the FAISS index
index = faiss.read_index("faiss_index.pkl")

# Load the docstore and index_to_docstore_id
with open("faiss_store.pkl", "rb") as f:
    docstore_dict, index_to_docstore_id = pickle.load(f)

# Convert the docstore dict to an InMemoryDocstore
# FAISS expects Docstore object instead of dict to use methods like search()
docstore = InMemoryDocstore(docstore_dict)

# Initialize the embedding function
embeddings = OpenAIEmbeddings(openai_api_key=API_KEY)

# Recreate the FAISS object
vectorstore = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=docstore,
    index_to_docstore_id=index_to_docstore_id
)

#### Agents

Agents use an LLM to determine which actions to perform and in what order. An action can be either using a tool and observing its output or returning it to the user. 

#### Tools

Tools are functions that agents can use to interact with the world. These tools can be common utilities (e.g. search, math etc.), other chains, or even other agents.

In [4]:
from langchain.agents import load_tools
from langchain_openai import ChatOpenAI

# initialize LLM
llm = ChatOpenAI(openai_api_key=API_KEY, model_name="gpt-3.5-turbo", temperature=0.7)


# built-in tool in LangChain for math calc.
tool_names = ["llm-math"]
# math tool is made available to llm to use
tools = load_tools(tool_names, llm=llm)
tools

[Tool(name='Calculator', description='Useful for when you need to answer questions about math.', func=<bound method Chain.run of LLMMathChain(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['question'], template='Translate a math problem into a expression that can be executed using Python\'s numexpr library. Use the output of running this code to answer the question.\n\nQuestion: ${{Question with math problem.}}\n```text\n${{single line mathematical expression that solves the problem}}\n```\n...numexpr.evaluate(text)...\n```output\n${{Output of running the code}}\n```\nAnswer: ${{Answer}}\n\nBegin.\n\nQuestion: What is 37593 * 67?\n```text\n37593 * 67\n```\n...numexpr.evaluate("37593 * 67")...\n```output\n2518731\n```\nAnswer: 2518731\n\nQuestion: 37593^(1/5)\n```text\n37593**(1/5)\n```\n...numexpr.evaluate("37593**(1/5)")...\n```output\n8.222831614237718\n```\nAnswer: 8.222831614237718\n\nQuestion: {question}\n'), llm=ChatOpenAI(client=<openai.resources.chat.completions.Comp

In [5]:
from langchain.agents import Tool
# our toolbox
tool_list = [
    Tool(
        name = "Math Tool",
        # run method of the first and only tool loaded "llm_math"
        func=tools[0].run,
        description="Tool to calculate, nothing else"
    ),
]

In [6]:
from langchain.agents import initialize_agent

# creates an agent, it uses tool_list, llm, agent type which can use tools without prior examples
agent = initialize_agent(tool_list, 
                         llm, 
                         agent="zero-shot-react-description", 
                         verbose=True,
                         )
agent.run("How are you? Can you calculate for me?")

c:\Users\shahi\AppData\Local\Programs\Python\Python312\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(
c:\Users\shahi\AppData\Local\Programs\Python\Python312\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
This question doesn't require a calculation, so I'll just respond with an answer.
Action: 
Final Answer: I am here to assist you with any calculations you need.

> Finished chain.


'I am here to assist you with any calculations you need.'

In [7]:
agent.run("What is 100 devided by 25?")




> Entering new AgentExecutor chain...
I should use the Math Tool to calculate this division.
Action: Math Tool
Action Input: {'expression': '100/25'}
Observation: Answer: 4.0
Thought:I now know the final answer.
Final Answer: 4.0

> Finished chain.


'4.0'

#### Custom Tools

You can also create your own tools by creating a class that inherits from BaseTool

In [8]:
from typing import Optional
from langchain.tools import BaseTool
from langchain.callbacks.manager import AsyncCallbackManagerForToolRun, CallbackManagerForToolRun
from pydantic import Field

# creates a new class that inherits from BaseTool.
# BaseTool class allows creating custom tools.
class CustomSearchTool(BaseTool):

    # properties for agent to understand when and how to use this tool
    name = "restaurant search"
    description = "useful for when you need to answer questions about our restaurant"

    # pydantic uses field declaration to validate that the passed object is a FAISS vectorstore
    # and assigns it to the vectorstore attribute of the CustomSearchTool
    vectorstore: FAISS = Field(description="Vector store for restaurant information")

    # method defines what happens when the tool is used. It takes a query string and
    # optional run manager as arguments.
    def _run(self, query: str, run_manager: Optional[CallbackManagerForToolRun] = None) -> str:
        docs = self.vectorstore.similarity_search(query, k=4)  # Get top 4 relevant documents

        # extracts the content from each retrieved document and joins them into a single string
        # separated by new lines
        text_list = [doc.page_content for doc in docs]
        return "\n".join(text_list)
    
    # async method is a placeholder for an asynchronous version of the tool.
    # it's not used and will raise error if called.
    async def _arun(self, query: str, run_manager: Optional[AsyncCallbackManagerForToolRun] = None) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("custom_search does not support async")

In [9]:
from langchain.agents import AgentType
# vectorstore attribute is set when creating instance of CustomSearchTool
tools = [CustomSearchTool(vectorstore=vectorstore)]
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [10]:
agent.run("When does the restaurant open?") 



> Entering new AgentExecutor chain...
I should use the restaurant search tool to find the opening hours of the restaurant.
Action: restaurant search
Action Input: "restaurant hours"
Observation: Q: What are the hours of operation for your restaurant?
A: Our restaurant is open from 11 a.m. to 10 p.m. from Monday to Saturday. On Sundays, we open at 12 p.m. and close at 9 p.m.

Q: What type of cuisine does your restaurant serve?
A: Our restaurant specializes in contemporary American cuisine with an emphasis on local and sustainable ingredients.
Q: Do you have any specials or promotions running currently?
A: Yes, we have a Happy Hour promotion from 3 to 5 p.m. on weekdays, with special prices on selected appetizers and drinks.
Q: What steps is your restaurant taking to ensure safety amid the ongoing pandemic?
A: We adhere to strict health and safety protocols, including regular sanitization, maintaining physical distance between tables, and providing hand sanitizers for customers. We als

'The restaurant opens at 11 a.m. to 10 p.m. from Monday to Saturday, and 12 p.m. to 9 p.m. on Sundays.'